In [1]:
import time
import statistics as stats
import multiprocessing
import time 
import random
import time
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
start_program = time.time()
from multiprocessing import *
import pandas as pd
import numpy as np
from threading import Thread
import time
import subprocess 


In [2]:
results = [[] for i in range(5)]

n_it = 100


n_st = number_stations = 10
n_s = number_sensors = 100

def_st = 1
def_s = 3
def_r = 'day'


set_date = [random.random() for i in range(500)]


In [3]:

def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))



def random_date(start, end, prop, dform = '%Y-%m-%dT%H:%M:%S'):
    return str_time_prop(start, end, dform, prop)
    
    
def to_pm(v):
    return str(round(v[0][0],2)) + "$" + '\\' + "pm$" + str(round(v[1][0],2))


# Queries

In [4]:
iw_q1 = """
    select time, id_station, <sid> FROM "<db>"."autogen"."sensor" 
    where <stid> 
    AND time > '<timestamp>Z' - <nb><rangesUnit> 
    AND  time < '<timestamp>Z'
"""

iw_q2 = """
    select time, id_station, <sid> FROM "<db>"."autogen"."sensor" 
    where <stid> 
    AND time > '<timestamp>Z' - <nb><rangesUnit> 
    AND  time < '<timestamp>Z' 
    and <sfilter>
""" #s<sid>

iw_q3 = """
    SELECT <avg_s> FROM "<db>"."autogen"."sensor" 
    WHERE  time > '<timestamp>Z' - <nb><rangesUnit> 
    AND time < '<timestamp>Z' 
    and <stid> 
    GROUP BY "id_station"  
"""  #mean(s<sid>)

iw_q4 = """
    SELECT first(id_station), <avg_s> FROM "<db>"."autogen"."sensor" 
    WHERE time > '<timestamp>Z' - <nb><rangesUnit> 
    and time < '<timestamp>Z' 
    and <stid> 
    GROUP BY id_station,time(1h)
""" #mean(s<sid>)
iw_q5 = """
    SELECT id_station, mean_value FROM (SELECT <avg_s> as mean_value FROM "<db>"."autogen"."sensor" 
    WHERE time > '<timestamp>Z' - <nb><rangesUnit> 
    AND time < '<timestamp>Z' 
    and <stid> 
    GROUP BY id_station,time(5s) FILL(0)) 
    GROUP BY id_station
""" #mean(s<sid>)


In [5]:
from influxdb import InfluxDBClient
from tqdm import tqdm 
import time

class Influx_Wide:
 
    # A sample method 
    @staticmethod
    def query(query, db, max_d, rangesUnit, n_it, start_date = "2019-04-01T00:00:00", stop_date = "2019-04-30T00:00:00", n_st = 1, n_s = 10):
        client = InfluxDBClient(host="diufrm118", port=8086, username='abdel')
        results = [[],[]]
        
        duration = max_d
        runtimes = []
        full_time = time.time()
        for i in tqdm(range(n_it)):
            date = random_date(start_date, stop_date, set_date[(duration*i)%500], dform = '%Y-%m-%dT%H:%M:%S')
            temp = query.replace("<timestamp>", date)
            temp = temp.replace("<db>", db)
            temp = temp.replace("<nb>", str(duration))
            temp = temp.replace("<rangesUnit>", str(rangesUnit[0]))
#             temp = temp.replace("<sid>", str(set_s[(duration*i)%500]))
            
            
            
            li = ['st' + str(z) for z in random.sample(range(number_stations), n_st)]
#                     print(li)
            q = '(id_station =' + "'" + li[0] + "'"
            for j in li[1:]:
                q += ' OR '  + 'id_station =' + "'" + j + "'"
            q += ")"
            temp = temp.replace("<stid>", q)
            
            
            # sensor
            li = ['s' + str(z) for z in random.sample(range(number_sensors), n_s)]
            q = li[0]
            q_filter = "( " + li[0] + ' > 0.95'
            q_avg = 'mean(' + li[0] + ')'
            for j in li[1:]:
                q += ', ' + j
#                 q_filter += ' OR ' + j + ' > 0.95'
                q_avg += ', ' + 'mean(' + j + ')'
            temp = temp.replace("<sid>", q)
            temp = temp.replace("<sfilter>", q_filter + ")")
            temp = temp.replace("<avg_s>", q_avg)                
            
            
            start = time.time()
#             print(temp)
            result = client.query(temp)
            diff = (time.time()-start)*1000
#             print(temp, diff)
            runtimes.append(diff)
            if time.time() - full_time > 20 and i > 5: 
                break              
#         print(temp)
#             print(runtimes)
        results[0].append(stats.mean(runtimes))
#             results[1].append(percentile(runtimes,85))
        results[1].append(stats.stdev(runtimes))
        client.close()
        return results
    
    @staticmethod
    def insert_file(batch_size,rate, st):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'
        template_sql = "sensor,id_station=st" + str(st) + " "

        client = InfluxDBClient(host='diufrm118', port=8086, username='abdel', database='d1_wide')
        file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
        f = open(file, "r")
        f = f.read()
        data = []
        f = f.splitlines()[1:]
#         f = ["\n".join(f[i:i+rate])  for i in range(0, len(f), rate)]
        for i in range(len(f)):
            f[i] = f[i].split(',')            
            temp = template_sql
            temp += 's0=' + f[i][2]
            cpt = 1
            for e in f[i][3:]:
                temp += ",s" + str(cpt) + "=" + str(e)
                cpt += 1
            temp += " " + str(10**3*int(datetime.timestamp( datetime.strptime(f[i][0], '%Y/%m/%dT%H:%M:%S'))))
            f[i] = temp        
        
        start = time.time()
        a = 0
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
#             print(f[a*rate:(1+a)*rate])
            client.write_points(f[a*rate:(1+a)*rate], database='d1_wide', time_precision='ms', batch_size=rate, protocol='line')
            a += 1
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)
                      
        
    @staticmethod
    def run_all(start, stop, db):
        results[0].append(to_pm(Influx_Wide.query(iw_q1, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[1].append(to_pm(Influx_Wide.query(iw_q2, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[2].append(to_pm(Influx_Wide.query(iw_q3, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[3].append(to_pm(Influx_Wide.query(iw_q4, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[4].append(to_pm(Influx_Wide.query(iw_q5, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
    
    
    
    

In [6]:
Influx_Wide.run_all("2019-04-01T00:00:00", "2019-04-30T00:00:00", "d1_wide")

 71%|██████████████████████         | 71/100 [00:20<00:08,  3.52it/s]


In [ ]:
stop_insertion = [False for i in range(1)]
Influx_Wide.insert_file(100_000, 2, 0)

In [ ]:
n_threads = 10
stop_insertion = [False for i in range(n_threads)]
rate = 200

# 20K dp/s

# Launch insertion
for i in range(n_threads):  
    t1 = Thread(target = Influx_Wide.insert_file, args=(100_000, rate, i))
#     t1 = Thread(target = ClickHouse.insert_by_point, args=(rate, i))
    t1.setDaemon(True)
    t1.start()

# insert_new(200, 1)

In [7]:
time.sleep(600)

In [8]:
Influx_Wide.run_all("2019-04-01T00:00:00", "2019-04-30T00:00:00", "d1_wide")

 63%|███████████████████▌           | 63/100 [00:20<00:11,  3.14it/s]


In [9]:
Influx_Wide.run_all("2019-05-01T01:00:00", "2019-05-02T00:00:00", "d1_wide")

 63%|███████████████████▌           | 63/100 [00:20<00:11,  3.15it/s]


In [10]:
Influx_Wide.run_all("2019-04-29T01:00:00", "2019-05-02T00:00:00", "d1_wide")

 60%|██████████████████▌            | 60/100 [00:20<00:13,  2.96it/s]


In [ ]:
stop_insertion = [True for i in range(n_threads)]


In [11]:
results 

[['90.51$\\pm$11.1',
  '103.62$\\pm$13.32',
  '102.86$\\pm$13.52',
  '104.45$\\pm$13.6'],
 ['17.37$\\pm$2.78',
  '79.04$\\pm$92.04',
  '92.42$\\pm$98.24',
  '51.27$\\pm$68.28'],
 ['14.57$\\pm$2.66',
  '58.46$\\pm$79.41',
  '52.57$\\pm$72.71',
  '40.19$\\pm$54.68'],
 ['15.76$\\pm$2.8', '49.72$\\pm$67.71', '64.3$\\pm$81.91', '39.37$\\pm$51.23'],
 ['279.53$\\pm$24.18',
  '312.48$\\pm$39.93',
  '311.55$\\pm$36.18',
  '331.15$\\pm$141.41']]

In [ ]:
runtimes = [[float(results[i][j].split('$')[0]) for j in range(len(results[i]))][:4] for i in range(5)]
variance = [[float(results[i][j].split('$')[2]) for j in range(len(results[i]))][:4] for i in range(5)]

In [ ]:
runtimes = list(map(list, zip(*runtimes)))
variance = list(map(list, zip(*variance)))


In [ ]:
runtimes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.arange(5)
labels = ['query'+str(1+i) for i in range(5)]
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0,0,1,1])
for i in range(4):
    ax.bar(X +0.15*i, runtimes[i], yerr = variance[i], width = 0.15)
ax.legend(['offline', 'online old', 'online new', 'online mixed', ])
ax.set_xticklabels(labels)
ax.set_xticks(np.arange(len(labels)))
ax.yaxis.grid(True)
plt.title('ClickHouse offline vs. online, rate: '+ str(rate) + ' x ' + str(n_threads))
plt.tight_layout()

# ax.bar(X+0.25, data[1], color = 'g', width = 0.25)
# ax.bar(X+0.5, data[2], color = 'r', width = 0.25)

In [ ]:
from influxdb import InfluxDBClient
import time

start = time.time()
client = InfluxDBClient(host='diufrm118', port=8086, username='abdel', database='d1_wide')
result = client.query("""DELETE FROM "sensor" where time > '2019-04-30T00:00:00Z'""")
print(time.time() - start)

# result = client.query("select * into d1_wide_ingest.autogen.sensor FROM d1_wide.autogen.sensor")

